### 1.多单建模--新建label 1-4 4head

In [ ]:
# multi-order
self._ord_cnt_weights = tf.where(label_ord_cnt > 0, label_ord_cnt, tf.ones_like(label_ord_cnt))
self.clipped_ord_cnt = tf.clip_by_value(label_ord_cnt, 1, 5)
self.ord_gt1_label = tf.to_float(tf.greater(label_ord_cnt, 1))
self.ord_gt2_label = tf.to_float(tf.greater(label_ord_cnt, 2))
self.ord_gt3_label = tf.to_float(tf.greater(label_ord_cnt, 3))
self.ord_gt4_label = tf.to_float(tf.greater(label_ord_cnt, 4))


### 2. 多单建模-- 中间隐层

In [ ]:
# shape - (?, 64)
self._ord_cvr_net = self.build_dnn(net=self._cvr_dnn_net1, dims=[64, 64], name='Task_Ord_Tower')
self._ord_cvr_net = self.mark_tag(self._ord_cvr_net, "ord_cvr_net")


with tf.variable_scope('GMSL_ord_cvr'):
    with tf.variable_scope('ord_cvr_logits'):
        input_layer = tf.concat([self._ord_cvr_net, self._ctr_dnn_net1, self._cvr_dnn_net1], axis=1)
        self._ord_cvr_logits = layers.fully_connected(input_layer, 4, activation_fn=None)


### 3. 拟合不同层的label


In [ ]:
self.gate_nu_summary = OrderedDict()
self.gate_nu_summary['Hidden_{}_{}_GateNU'.format(layer_id, name)] = gate_nu

for name, var in self.gate_nu_summary.items():
    variable_summaries(model, name, var, mode)

self.gt1_pvp_prediction = tf.sigmoid(self._ord_cvr_logits[:,:1]) * self._pv_click_prediction
self.gt2_pvp_prediction = tf.sigmoid(self._ord_cvr_logits[:,1:2]) * self._pv_click_prediction
self.gt3_pvp_prediction = tf.sigmoid(self._ord_cvr_logits[:,2:3]) * self._pv_click_prediction
self.gt4_pvp_prediction = tf.sigmoid(self._ord_cvr_logits[:,3:4]) * self._pv_click_prediction
self._correct_gt1_pvp = tf.sigmoid(self._ord_cvr_logits[:, :1]) * self._correct_ctr_prediction
self._correct_gt2_pvp = tf.sigmoid(self._ord_cvr_logits[:, 1:2]) * self._correct_ctr_prediction
self._correct_gt3_pvp = tf.sigmoid(self._ord_cvr_logits[:, 2:3]) * self._correct_ctr_prediction
self._correct_gt4_pvp = tf.sigmoid(self._ord_cvr_logits[:, 3:4]) * self._correct_ctr_prediction

### 4. 联合loss

In [ ]:
self.LOSSES = {
    'pv_clk': LossFunc('bce')(y_true=self._click_labels, y_pred=self._pv_click_prediction, mask=self._pv_labels),
    'clk_pay': LossFunc('bce')(y_true=self._pay_labels, y_pred=self._click_pay_prediction,  mask=self._click_labels),
    'pv_pay': LossFunc('bce')(y_true=self._pay_labels, y_pred=self._mpvp,  mask=self._pv_labels),
    'gt1_pay': LossFunc('bce')(y_true=self.ord_gt1_label, y_pred=self.gt1_pvp_prediction,  mask=self._pv_labels),
    'gt2_pay': LossFunc('bce')(y_true=self.ord_gt2_label, y_pred=self.gt2_pvp_prediction,  mask=self._pv_labels),
    'gt3_pay': LossFunc('bce')(y_true=self.ord_gt3_label, y_pred=self.gt3_pvp_prediction,  mask=self._pv_labels),
    'gt4_pay': LossFunc('bce')(y_true=self.ord_gt4_label, y_pred=self.gt4_pvp_prediction,  mask=self._pv_labels),
    'reg': tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
}

self.LOSSES['loss'] = self._loss = self.LOSSES['pv_clk'] + self.LOSSES['clk_pay'] + self.LOSSES['pv_pay'] + \
                                    self.LOSSES['gt1_pay'] + self.LOSSES['gt2_pay'] + self.LOSSES['gt3_pay'] + self.LOSSES['gt4_pay'] + self.LOSSES['reg']

### 5. 计算AUC

In [ ]:
_, self._gt1_auc_op = calc_auc(label=self.ord_gt1_label, pred=self.gt1_pvp_prediction,
                                mask=self._pv_labels, is_training=self._is_training, name="gt1_auc")
_, self._gt2_auc_op = calc_auc(label=self.ord_gt2_label, pred=self.gt2_pvp_prediction,
                                mask=self._pv_labels, is_training=self._is_training, name="gt2_auc")
_, self._gt3_auc_op = calc_auc(label=self.ord_gt3_label, pred=self.gt3_pvp_prediction,
                                mask=self._pv_labels, is_training=self._is_training, name="gt3_auc")
_, self._gt4_auc_op = calc_auc(label=self.ord_gt4_label, pred=self.gt4_pvp_prediction,
                                mask=self._pv_labels, is_training=self._is_training, name="gt4_auc")


scalars = OrderedDict()
scalars["ctr_auc"] = self._ctr_auc_op
scalars["cali_ctr_auc"] = self._cali_ctr_auc_op
scalars["cvr_auc"] = self._cvr_auc_op
scalars["pvp_auc"] = self._pvp_auc_op
scalars["gt1_auc"] = self._gt1_auc_op
scalars["gt2_auc"] = self._gt2_auc_op
scalars["gt3_auc"] = self._gt3_auc_op
scalars["gt4_auc"] = self._gt4_auc_op
scalars["cali_pvp_auc"] = self._cali_pvp_auc_op
scalars["batch_reg_loss"] = self.LOSSES["reg"]
scalars["batch_ctr_loss"] = self.LOSSES["pv_clk"]
scalars["batch_cvr_loss"] = self.LOSSES["clk_pay"]
scalars["batch_pvp_loss"] = self.LOSSES["pv_pay"]
scalars["label_pcoc"] = tf.reduce_sum(self._pv_pay_prediction) / tf.reduce_sum(self._pay_labels)
scalars["ord_pcoc"] = tf.reduce_sum(self._pv_pay_prediction + self.gt1_pvp_prediction + self.gt2_pvp_prediction + self.gt3_pvp_prediction + self.gt4_pvp_prediction) / tf.reduce_sum(self.clipped_ord_cnt * self._pay_labels)
scalars["batch_loss"] = self._loss
scalars["batch_comp_loss"] = self._comp_loss
model.add_summary(scalars, type="scalar", mode=mode)

### 6. 统计公式

模型里引入多单head，增加了5个head，分别建模P(Order>=K),K=2,3,4,5的概率

绝对价格带	多单比例	pvp_pcoc	pvp12_pcoc	pvp123_pcoc	pvp1234_pcoc	pvp12345_pcoc	pv_pay_auc	pay12_auc	pay123_auc	pay1234_auc	pay12345_auc
[-w,0.5)	26.3%	0.595	0.753	0.830	0.874	0.944	0.8884	0.8878	0.8868	0.8862	0.885
[0.5,2.0)	12.5%	0.834	0.936	0.970	0.985	1.006	0.8833	0.8829	0.8825	0.8823	0.8819
[2.0,5.0)	6.4%	0.925	0.980	0.991	0.996	1.001	0.8968	0.8969	0.8968	0.8968	0.8968
[5.0,10.0)	4.1%	0.956	1.000	1.008	1.011	1.014	0.9082	0.9082	0.9082	0.9082	0.9082
[10.0,20.0)	4.5%	0.993	1.039	1.048	1.052	1.057	0.9219	0.9219	0.9219	0.9219	0.9219
[20.0,+w)	4.1%	0.902	0.945	0.953	0.957	0.963	0.9402	0.9402	0.9402	0.9402	0.9402


